# Installing Libraries

In [110]:
!pip install opencv-python
!pip install tensorflow
!pip install pytesseract
!pip install spacy

# Importing Packages

In [1]:
import cv2 
import matplotlib.pyplot as plt
import pytesseract
import shutil
import os
import random
import nltk
import numpy as np
import re
try:
 from PIL import Image
except ImportError:
 import Image

# Downloading Packages

In [121]:
!python3 -m spacy download en_core_web_sm

2022-07-18 12:30:37.290270: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mustafa/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-18 12:30:37.290299: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /home/mustafa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/mustafa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/mustafa/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/mustafa/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

# Pre Processing Image

In [2]:
def ZoomImage(img, zoom_factor=2):
    return cv2.resize(img, None, fx= zoom_factor, fy= zoom_factor, interpolation= cv2.INTER_LINEAR)

def GrayScaleImage(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = 255*(gray < 128).astype(np.uint8) 
    return gray

def remove_noise(img):
    return cv2.medianBlur(img,5)

def InvertImage(img):
    return cv2.bitwise_not(img)

def CropImage(img):
    # Find all non-zero points (text)
    coords = cv2.findNonZero(img) 
    # Find minimum spanning bounding box
    x, y, w, h = cv2.boundingRect(coords) 
    # Crop the image - note we do this on the original image
    img = img[y:y+h, x:x+w] 
    return img

# Creating Bounding Box In Image

In [3]:
def CreateBoundingBox(img):
    h, w, c = img.shape
    boxes = pytesseract.image_to_boxes(img) 
    for b in boxes.splitlines():
        b = b.split(' ')
        img = cv2.rectangle(img, (int(b[1]), h - int(b[2])), (int(b[3]), h - int(b[4])), (0, 255, 0), 2)
    return img

# Extracting Text From Image

In [4]:
def ExtractText(img) : 
    custom_config = r'--oem 3 --psm 6'
    text = pytesseract.image_to_string(img, config=custom_config)
    return text

# Remove Extra White Spaces

Removing Extra Spaces for Email

In [5]:
# from nltk.tokenize import sent_tokenize, word_tokenize
# # valid_characters = re.compile("[A-Za-z0-9]+")
# words = word_tokenize(text)
# for i in words:
# #     if '@' in i:
#     print(i)
# for i in range(len(text)):
#     if text[i] == '@':
#         while (not valid_characters.fullmatch(text[i])):
#             i = i - 1
#             if (text[i] == ' '):
                
#         print(text[i])
#         break

# Extracting Name

In [6]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')

ignore_text1 = "CURRICULUM"
ignore_text2 = "VITAE"

matcher = Matcher(nlp.vocab)

def extract_name(content):
    nlp_text = nlp(content)
    
    pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
    
    matcher.add('NAME', [pattern])
    
    matches = matcher(nlp_text)

    for match_id, start, end in matches:
        span = nlp_text[start:end]
        if not(ignore_text1.lower() in span.text.lower() or ignore_text2.lower() in span.text.lower()) :
            return span.text
        
    return ""

2022-07-19 19:48:09.917939: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/mustafa/.local/lib/python3.8/site-packages/cv2/../../lib64:
2022-07-19 19:48:09.917969: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Extracting Phone Number

In [7]:
PHONE_REG = re.compile(r'[\+\(]?[1-9][0-9 .\-\(\)]{8,}[0-9]')
def extract_phone_number(resume_text):
    phone = re.findall(PHONE_REG, resume_text)
    if phone:
        number = ''.join(phone[0])
 
        if resume_text.find(number) >= 0 and len(number) < 16:
            return number
    return None

# Extracting Email ID

In [8]:
EMAIL_REG = re.compile(r'[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z]+')

def extract_emails(resume_text):
    return re.findall(EMAIL_REG, resume_text)

# Main Program

In [ ]:
import csv
import glob

with open('output_top_10.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    for i in range(10):
        IMG_PATH = 'SampleResumes/' + str(i+1)
        text = ""
        img_list = []
        for filename in glob.glob(IMG_PATH + '/*.jpeg'): 
            
            # Img Pre Processing
            
            img = cv2.imread(filename)

            img = ZoomImage(img , 3)
            img = GrayScaleImage(img)
            img = CropImage(img)
            img = InvertImage(img)

            cv2.imwrite(IMG_PATH + '/0.png', img)
            img = cv2.imread(IMG_PATH + '/0.png')

            boundedImg = CreateBoundingBox(img)
            cv2.imwrite(IMG_PATH + '/a.png', boundedImg)
            
            img = cv2.imread(IMG_PATH + '/0.png')

            # NLP Part

            text = ExtractText(img)
            text = text + "\n"

        personName = extract_name(text.lower())
        contactNum = extract_phone_number(text.lower())
        emailID = extract_emails(text.lower())
        
        row = []
        row.append(personName)
        row.append(contactNum)
        row.append(emailID)
        print(row)
        
        writer.writerow(row)
        print(img_list)
#         break
    f.close()
        

['mubasheer ahmed', '+919182834870', ['mubasheerkhan@gmail.com']]
[]
['jumeirah beach', None, ['gppraveen19@gmail.com']]
[]


In [ ]:
|